In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('../database/intents3.json').read()
intents = json.loads(data_file)

for intent in intents['type3']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [3]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-3-7fa144a2511d>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [4]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.02, decay=1e-6, momentum=0.7, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)
model.save('../models/model3.h5', hist)

print("model created")

Epoch 1/100
19/19 [==============================] - 0s 1ms/step - loss: 3.4006 - accuracy: 0.0215
Epoch 2/100
19/19 [==============================] - 0s 825us/step - loss: 3.3347 - accuracy: 0.0538
Epoch 3/100
19/19 [==============================] - 0s 998us/step - loss: 3.2430 - accuracy: 0.1290
Epoch 4/100
19/19 [==============================] - 0s 971us/step - loss: 3.1170 - accuracy: 0.2043
Epoch 5/100
19/19 [==============================] - 0s 929us/step - loss: 3.1103 - accuracy: 0.1183
Epoch 6/100
19/19 [==============================] - 0s 825us/step - loss: 3.0499 - accuracy: 0.1398
Epoch 7/100
19/19 [==============================] - 0s 912us/step - loss: 2.9099 - accuracy: 0.1828
Epoch 8/100
19/19 [==============================] - 0s 988us/step - loss: 2.6984 - accuracy: 0.2796
Epoch 9/100
19/19 [==============================] - 0s 941us/step - loss: 2.7025 - accuracy: 0.3011
Epoch 10/100
19/19 [==============================] - 0s 973us/step - loss: 2.5460 - accuracy

19/19 [==============================] - 0s 1ms/step - loss: 0.3030 - accuracy: 0.9140
Epoch 83/100
19/19 [==============================] - 0s 918us/step - loss: 0.2084 - accuracy: 0.9570
Epoch 84/100
19/19 [==============================] - 0s 813us/step - loss: 0.2980 - accuracy: 0.9247
Epoch 85/100
19/19 [==============================] - 0s 815us/step - loss: 0.2227 - accuracy: 0.9247
Epoch 86/100
19/19 [==============================] - 0s 870us/step - loss: 0.1916 - accuracy: 0.9355
Epoch 87/100
19/19 [==============================] - 0s 857us/step - loss: 0.2186 - accuracy: 0.9247
Epoch 88/100
19/19 [==============================] - 0s 829us/step - loss: 0.2126 - accuracy: 0.9462
Epoch 89/100
19/19 [==============================] - 0s 820us/step - loss: 0.3599 - accuracy: 0.8710
Epoch 90/100
19/19 [==============================] - 0s 816us/step - loss: 0.3251 - accuracy: 0.8925
Epoch 91/100
19/19 [==============================] - 0s 868us/step - loss: 0.2334 - accuracy: 0.